<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="../media/inria-25-alpha.png" /></span>
</div>

# un push compliqué

scénario #2 :  
un push qui ne se passe pas bien

* un cas **très fréquent** en pratique
* et qui génère pas mal de frustration 

* deux personnes se mettent à travailler  
* sur des sujets différents
* mais en même temps  
  (i.e. en partant du même commit)

* ils commitent chacun de leur coté

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## une topologie réaliste

dans ce scénario plus réaliste :

* bob et alice on chacun leur dépôt
* et collaborent au travers d'un proxy sur github

on a donc ici :

* deux dépots "complets" avec fichiers
* un dépôt *bare* sur github

![](../media/archi-gh-3.png)

In [ ]:
# on repart d'un dépôt tout simple 
# avec seulement deux commits 
# pour ne pas encombrer inutilement l'affichage

# on nettoie
cd $TOP
rm -rf repo-alice fake-github.git repo-bob

In [ ]:
# on recrée repo-alice avec deux commits
cd $TOP
mkdir repo-alice
cd repo-alice
$SCRIPTS/do populate-repo-alice > /dev/null

In [ ]:
# on crée un dépôt bare qui remplace github
# pour faire le proxy entre les deux acteurs
cd $TOP
git clone --bare repo-alice fake-github.git

# on clone le faux github dans repo-bob
cd $TOP
git clone fake-github.git repo-bob

les trois repos sont en phase

In [ ]:
cd $TOP/repo-alice
git l

In [ ]:
cd $TOP/fake-github.git
git l

In [ ]:
cd $TOP/repo-bob
git l

## le `push` qui tue (1)

In [ ]:
# alice avance de son coté

cd $TOP/repo-alice
$SCRIPTS/do commit-alice

git l

In [ ]:
# bob aussi

cd $TOP/repo-bob
$SCRIPTS/do commit-bob

git l

**ici aussi il nous faut un remote pour pousser**

à ce stade les deux acteurs vont vouloir pousser leur travail sur github  
pour ça il leur faut un remote

In [ ]:
# dans repo-alice on ne connait aucun remote
# on en crée un qui s'appelle github

cd $TOP/repo-alice
git remote add github $TOP/fake-github.git
git remote

In [ ]:
# on a déjà un remote 'origin' lié au clone initial
# mais on l'ignore et on crée un second remote
# pour homogénéité entre les deux repos d'alice et de bob 

cd $TOP/repo-bob
git remote add github $TOP/fake-github.git
git remote

## le `push` qui tue (2)

le premier des deux qui veut pousser sur github n'a aucun problème

In [ ]:
cd $TOP/repo-alice

git push github devel:devel 

In [ ]:
git l

In [ ]:
cd $TOP/fake-github.git
git l

## le `push` qui tue (3)

* bob veut pousser lui aussi
* c'est à ce stade que ça coince
* car ce push implique un merge
* qui n'**est pas *fast-forward***
* en effet le graphe des commits ressemble à ceci

```
devel pour alice et github        devel pour bob
                     ↳   A      B   ↵
                           \   /
                             C2
                             |
                             C1
```

* un `push` de la part de bob 
* revient donc à merger `B` au dessus de `A`
* qui comme on le voit **ne sont pas comparables**

## le `push` qui tue (4)

In [ ]:
# si bob essaie de pousser à ce stade, c'est refusé
# car le merge, qui n'est pas fast-forward, impliquerait
# la création d'un nouveau commit, ce qui
# est risqué à distance 
#
# notez bien qu'ici les deux modifications de alice et bob
# sont indépendantes et peuvent être mergées sans conflit !

cd $TOP/repo-bob
git push github devel

## tirer avant de pousser

In [ ]:
# pour s'en sortir il suffit que Bob commence par tirer
# et c'est en tirant qu'on va créer le commit qui merge les deux travaux
# 
# pour des raisons sordides liées au fait qu'on est dans un notebook
# je lui passe l'option --no-edit

cd $TOP/repo-bob
git pull --no-edit github devel 

In [ ]:
git l

## le `push` ne tue plus

```
                                M   ←  devel pour bob
devel pour alice et github    /   \    
                        ↳   A      B  
                              \   /
                                C2
                                |
                                C1
```


pour Bob à présent, le fait de pousser `devel` sur `github` est redevenu un *fast-forward*, il peut pousser

Remarquez que dans ce cas de figure, Bob aurait pu profitablement tirer avec l'option `--rebase`, que l'on verra plus tard, mais qui aurait le mérite d'éviter cette structure en diamant qui n'est pas très significative. 

In [ ]:
cd $TOP/repo-bob
git push github devel

In [ ]:

git l

In [ ]:
# et alice peut tirer
cd $TOP/repo-alice
git pull github devel

In [ ]:
git l

## résumé

* si lors d'un `push` vous obtenez un message similaire à 

> hint: Updates were rejected because the remote contains work that you do
> hint: not have locally. This is usually caused by another repository pushing
> hint: to the same ref. You may want to first integrate the remote changes
> hint: (e.g., 'git pull ...') before pushing again.

* c'est vraisemblablement parce que quelqu'un d'autre a poussé 
  avant vous
 
* un moyen simple de s'en sortir consiste à tirer d'abord  
  et pousser ensuite
  
* signalons aussi l'option `pull --rebase` que l'on verra plus tard

## état

In [ ]:
git l --all